In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install lemminflect
!pip install tomotopy

In [13]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import tomotopy as tp
from tqdm import tqdm
from nltk import pos_tag
from nltk.tokenize import word_tokenize
# nltk.download()
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
import spacy
import lemminflect
from lemminflect import getLemma
from operator import itemgetter
import sys
import os
import re
import collections
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers.rnn import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout, Embedding
from keras.layers.normalization import batch_normalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.53.216.210:8470


REPLICAS:  8


In [ ]:
stops = stopwords.words('english')
newstop = ['et', 'al', 'us','factor','evidence','review','analysis','impact','effect','example','research','paper','result',"first","second","third","forth",'doe', 'C', 'elsevier', 'ltd', 'etc', 'was', 'wa', 'ha', 'springer','mdpi','evaluation','evaluating']
stops.extend(newstop)
path = '/content/drive/MyDrive/0809'
os.chdir(path)

In [41]:
content = pd.read_excel('0809_cc_train_token.xlsx')
train = content[['Abstract','category','toxic','token']]
train

,Abstract,category,toxic,token
0,"Fossil fuel consumption, luxurious lifestyles,...",11,energy,fossil fuel consumption luxurious lifestyle po...
1,Global climate change is a change in the long-...,11,energy,global climate-change change long-term weather...
2,Many policy-makers view nuclear power as a mit...,11,energy,many policy-maker view nuclear power mitigatio...
3,"The climate varies on multiple timescales, but...",11,energy,climate vary multiple timescale human main age...
4,Innovative digital twins (DTs) that allow engi...,11,energy,innovative digital twin Dt engineer visualise ...
...,...,...,...,...
15088,It is of importance to realize high-efficiency...,24,heat island,importance high-efficiency clean utilization f...
15089,The co-firing of various ultra-low volatile co...,24,heat island,co-firing various ultra-low volatile coal-base...
15090,The deployment of CO2 capture and storage (or ...,24,heat island,deployment CO2 capture storage carbon capture ...
15091,There are a few economy and power industry dec...,24,heat island,economy power industry decarbonization line co...


In [42]:
train[train['token'].isna()]

,Abstract,category,toxic,token
11694,.,16,land-use,NaN


In [43]:
train = train.dropna(axis=0)
train

,Abstract,category,toxic,token
0,"Fossil fuel consumption, luxurious lifestyles,...",11,energy,fossil fuel consumption luxurious lifestyle po...
1,Global climate change is a change in the long-...,11,energy,global climate-change change long-term weather...
2,Many policy-makers view nuclear power as a mit...,11,energy,many policy-maker view nuclear power mitigatio...
3,"The climate varies on multiple timescales, but...",11,energy,climate vary multiple timescale human main age...
4,Innovative digital twins (DTs) that allow engi...,11,energy,innovative digital twin Dt engineer visualise ...
...,...,...,...,...
15088,It is of importance to realize high-efficiency...,24,heat island,importance high-efficiency clean utilization f...
15089,The co-firing of various ultra-low volatile co...,24,heat island,co-firing various ultra-low volatile coal-base...
15090,The deployment of CO2 capture and storage (or ...,24,heat island,deployment CO2 capture storage carbon capture ...
15091,There are a few economy and power industry dec...,24,heat island,economy power industry decarbonization line co...


In [23]:
sorted(train.category.unique())

[11, 12, 13, 14, 15, 16, 21, 22, 23, 24]

In [46]:
print(train['token'].apply(lambda x:len(str(x).split())).max())
max_len = (round(train['token'].apply(lambda x:len(str(x).split())).max()/100)*100)
print(max_len)

358
400


In [45]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.token.values, train.category.values,
                                                  stratify=train.category.values,
                                                  random_state=42,
                                                  test_size=0.2, shuffle=True)

In [47]:
xtrain

array(['eco-city project prevalent globe new urban environment sustainable urban environmental-economic transition low-carbon technological industrial base article argue need thermal aspect urban metabolism time individual building eco-city master plan economic development strategy state level article critical eco-city design focus role specific building structure eco-city political ecology scale stretch specific climatic characteristic metabolic master plan eco-city provincial regional state-level plan integration eco-city wider economic political development trajectory article focus masdar abu dhabus eco-city construction time',
       'anticipate challenge wind industry standardize wind infrastructure expiration respective approval period energy climate target german federal government challenge context sustainability potential concept deconstruction infrastructure replacement ii operation plant iii newer modernize plant re-powering re-powering infrastructure innovative technology h

In [48]:
xvalid

array(['urban heat island UHU heat foster biophysical hazard heat stress air pollution public health problem mitigation strategy vegetative cover higher albedo material atlantum georgia extreme heat better understand heat island relate weather modification objective characterize temporal variation magnitude UHU metro atlantum area climatological attribute UHU high temperature condition atlantum summer june july august period conduct theoretical numerical simulation first-order mitigation strategy period 1984-2007 climatological mean UHU magnitude atlanta-athen athens-monticello 71UM degree significant minimum temperature trend 70UM degree decade athen 79UM degree decade monticello atlantum minimum temperature unchanged largest smallest UHU magnitude spring summer cloud-radiative cycle heat atlantum year 1984-2007 summertime phenomenon mean number heat event atlantum heat year average atlantum heat wave day bit variability standard deviation mean maximum temperature atlantum heat 85UM d

In [49]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
# max_len = 1000

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [50]:
# Calculating the Vocabulary Size and the number of Answers

print("Vocabulary size={}".format(len(token.word_index)))
print("Number of Answers={}".format(token.document_count))

Vocabulary size=34260
Number of Answers=15092


In [51]:
le = LabelEncoder()
num_classes=len(train.category.unique()) # 1 = Society & Culture 2 = Science & Mathematics 3 = Health 4 = Education & Reference 5 = Computers & Internet 6 = Sports 7 = Business & Finance 8 = Entertainment & Music 9 = Family & Relationships 10 = Politics & Government

ytrain = le.fit_transform(ytrain)
yvalid = le.transform(yvalid)

ytrain = to_categorical(ytrain)
yvalid = to_categorical(yvalid)

In [52]:
ytrain

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [53]:
ytrain.shape

(12073, 10)

# Word Embeddings

While building our simple RNN models we talked about using word-embeddings , So what is word-embeddings and how do we get word-embeddings?
Here is the answer :
* https://www.coursera.org/learn/nlp-sequence-models/lecture/6Oq70/word-representation
* https://machinelearningmastery.com/what-are-word-embeddings/
<br> <br>
The latest approach to getting word Embeddings is using pretained GLoVe or using Fasttext. Without going into too much details, I would explain how to create sentence vectors and how can we use them to create a machine learning model on top of it and since I am a fan of GloVe vectors, word2vec and fasttext. In this Notebook, I'll be using the GloVe vectors. You can download the GloVe vectors from here http://www-nlp.stanford.edu/data/glove.840B.300d.zip or you can search for GloVe in datasets on Kaggle and add the file

In [55]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('glove.6B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

400001it [01:07, 5923.51it/s]

Found 400001 word vectors.


# LSTM's

## Basic Overview

Simple RNN's were certainly better than classical ML algorithms and gave state of the art results, but it failed to capture long term dependencies that is present in sentences . So in 1998-99 LSTM's were introduced to counter to these drawbacks.

## In Depth Understanding

Why LSTM's?
* https://www.coursera.org/learn/nlp-sequence-models/lecture/PKMRR/vanishing-gradients-with-rnns
* https://www.analyticsvidhya.com/blog/2017/12/fundamentals-of-deep-learning-introduction-to-lstm/

What are LSTM's?
* https://www.coursera.org/learn/nlp-sequence-models/lecture/KXoay/long-short-term-memory-lstm
* https://distill.pub/2019/memorization-in-rnns/
* https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

# Code Implementation

We have already tokenized and paded our text for input to LSTM's

In [56]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300)) # 300은 차원 수
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 34260/34260 [00:00<00:00, 109987.63it/s]


LSTM(n, input_dim = k, input_length = k)

n : 메모리 셀의 개수(기억용량, 출력형태를 결정함)

In [57]:
# %%time
with strategy.scope():

    # A simple LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

    model.add(LSTM(256, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='log_softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 300)          10278300  
                                                                 
 lstm (LSTM)                 (None, 256)               570368    
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 10,882,854
Trainable params: 604,554
Non-trainable params: 10,278,300
_________________________________________________________________


In [ ]:
model.fit(xtrain_pad, ytrain, validation_data = (xvalid_pad, yvalid), epochs=20, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/20
24/24 [==============================] - 17s 398ms/step - loss: 1.9363 - accuracy: 0.0222 - val_loss: 1.7618 - val_accuracy: 0.0126
Epoch 2/20
24/24 [==============================] - 3s 130ms/step - loss: 1.6912 - accuracy: 0.0084 - val_loss: 1.6421 - val_accuracy: 0.0053
Epoch 3/20
24/24 [==============================] - ETA: 0s - loss: 1.5975 - accuracy: 0.0035

In [ ]:
scores = model.evaluate(xvalid_pad, yvalid)
print("Accuracy: %.2f%%" % (scores[1]*100))

149/149 [==============================] - 315s 2s/step - loss: 0.1207 - accuracy: 0.6306
Accuracy: 63.06%


In [ ]:
model.save('15_LSTM_02.h5', save_format='h5')
model.save('15_LSTM_02.keras')

In [ ]:
import pickle
pickle.dump(model, open('15_LSTM_02.pkl', 'wb'))

In [ ]:
yvalid_model = model.predict(xvalid_pad)
yvalid_model

In [ ]:
print(xvalid[12])
print(np.where(scores[11] == np.max(scores[11])))
print(np.where(yvalid[11] == np.max(yvalid[11])))

multiple anthropogenic stressor co-occur natural ecosystem multiple stressor study nature direction stressor interaction strength stressor examine coral alpha beta-diversity vary site gradient chronic local anthropogenic stress marine heatwave multiple stressor framework encompass non-discrete stressor examine interaction continuou discrete stressor additive antagonistic interaction heatwave-driven turnover coral community composition continuou stressor point response coral hill-richness stressor additive synergistic community-level response multiple stressor vary stressor intensity importance complex realistic continuou stressor understand stressor interaction ecological
(array([7], dtype=int64),)
(array([5], dtype=int64),)
